In [1]:
from data_wrangler import synthetic_data_gen, feature_extractor
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from model import atp_pipeline, atp_graph, losses
from data_wrangler import dataset_preparer
from data_wrangler.batcher import batcher, batcher_np
import os
import IPython
import matplotlib.pyplot as plt
import numpy as np

In [2]:
exchange = pd.read_csv('datasets/exchange.csv')

In [3]:
array = np.array(exchange)
array[:,0] = np.linspace(-1,1,array.shape[0])
array = array.astype("float32")

In [4]:
array[:5000, :].shape

(5000, 9)

In [5]:
exchange

,date,0,1,2,3,4,5,6,OT
0,1990/1/1 0:00,0.785500,1.611000,0.861698,0.634196,0.211242,0.006838,0.525486,0.593000
1,1990/1/2 0:00,0.781800,1.610000,0.861104,0.633513,0.211242,0.006863,0.523972,0.594000
2,1990/1/3 0:00,0.786700,1.629300,0.861030,0.648508,0.211242,0.006975,0.526316,0.597300
3,1990/1/4 0:00,0.786000,1.637000,0.862069,0.650618,0.211242,0.006953,0.523834,0.597000
4,1990/1/5 0:00,0.784900,1.653000,0.861995,0.656254,0.211242,0.006940,0.527426,0.598500
...,...,...,...,...,...,...,...,...,...
7583,2010/10/6 0:00,0.718494,1.222195,0.737485,0.969974,0.143697,0.008500,0.688565,0.690846
7584,2010/10/7 0:00,0.721839,1.223459,0.741155,0.977297,0.143763,0.008595,0.690288,0.695701
7585,2010/10/8 0:00,0.723197,1.234111,0.745184,0.984446,0.143997,0.008562,0.691419,0.695943
7586,2010/10/9 0:00,0.720825,1.233905,0.744131,0.980344,0.143993,0.008555,0.690942,0.692689


In [6]:
def batcher_bc(array, n_C, n_T, batch_size=32):
    t = []
    y = []
    for i in range(batch_size):
        index = int(np.random.randint(0, array.shape[0] - n_C - n_T, 1))
        t_context =  array[index:index + n_C , 0]
        t_context = np.repeat(t_context, array.shape[1] - 1)[:, np.newaxis]
        t_target = array[index + n_C:index + n_C + n_T, 0]
        t_target = np.repeat(t_target, array.shape[1] - 1)[:, np.newaxis]
        
        y_temp_context = array[index:index + n_C , 1]
        y_temp_target = array[index + n_C:index + n_C + n_T, 1]
        y_temp_all = np.concatenate([y_temp_context, y_temp_target], axis=0)
        t_all = np.concatenate([t_context, t_target], axis=0)
        for i in range(2, array.shape[1]):
            y_temp_all = np.concatenate([y_temp_all, array[index:index + n_C + n_T, i]], axis=0)

        t.append(t_all)
        y.append(y_temp_all)
    return np.array(y), np.array(t)

In [7]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset_preparer.dataset_processor("datasets/exchange.csv") 


In [8]:
y_train.shape

(5235, 1, 1)

In [9]:
save_dir = 'weights/forecasting/exchange/multivariate'

In [15]:

run = 2
tr_step = atp_graph.build_graph()
model = atp_pipeline.atp_pipeline(multiply=8)
###### can we put the name of the model into the folder name #########?

name_comp = 'run_' + str(run)
folder = save_dir + '/ckpt/check_' + name_comp
if not os.path.exists(folder): os.mkdir(folder)
opt = tf.keras.optimizers.Adam(3e-4)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint) 
sum_mse_tot = 0; sum_nll_tot = 0
mini = 50000

In [16]:
n_C, n_T = 96, 96

In [17]:
tr_loss = [] ; val_loss = []
n_C2 = n_C * 8
n_T2 = n_T * 8
batch_s = 32
for i in range(2000):
    x, y = batcher_bc(array[:5000, :], n_C, n_T) 
    y_re = np.concatenate([y[:, n_C2 + n_T * i:n_C2+(n_T * (i +1))] for i in range(8)], axis=-1)
    y_re = y_re.reshape(batch_s, -1, 1)
    _,_, nll_pp_tr, _ = tr_step(model, opt, x[:, :, np.newaxis], y, n_C, n_T, training=True, multiple=8, y_re=y_re)
    
    
    if (i+1) % 100 == 0:
        x, y = batcher_bc(array[5000:6000, :], n_C, n_T) 
        y_re = np.concatenate([y[:, n_C2 + n_T * i:n_C2+(n_T * (i +1))] for i in range(8)], axis=-1)
        y_re = y_re.reshape(batch_s, -1, 1)
        mu, log_sigma = model([x[:, :, np.newaxis], y, n_C, n_T, False]) 
        _, _, _, nll_pp_val, mse = losses.nll(y_re, mu, log_sigma)
        if nll_pp_val.numpy() < mini:
            ckpt.step.assign_add(1)
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
            print("loss {:1.2f}".format(nll_pp_tr.numpy()))
        tr_loss.append(nll_pp_tr.numpy())
        val_loss.append(nll_pp_val.numpy())
        IPython.display.clear_output(wait=True)
        plt.plot(tr_loss)
        plt.plot(val_loss)
        plt.show()

2023-05-11 15:01:53.531401: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [22]:
mse

<tf.Tensor: shape=(), dtype=float32, numpy=9.765955e-05>